# **Exploration of COVID-19 Infection rates in NYC**


## NYC OpenData COVID-19 Infections
---
To explore the impact of COVID-19 vaccinations on the infection rate, we can first take a first look at the infection rate before and after
the vaccinations were available to the general public. Luckily for us, NYC OpenData has publicly available data on COVID-19 infections. This link to the data can be found [here](https://data.cityofnewyork.us/Health/COVID-19-Outcomes-by-Testing-Cohorts-Cases-Hospita/cwmx-mvra).
Using pandas and reading the csv, your dataframe should look something like this:

In [40]:
import pandas as pd
covid_infections = pd.read_csv("covid_cases.csv")
covid_infections.head()

,extract_date,specimen_date,Number_tested,Number_confirmed,Number_hospitalized,Number_deaths
0,04/29/2020,04/17/2020,9979,3386,527,96
1,04/29/2020,02/08/2020,1,0,0,0
2,04/29/2020,03/05/2020,63,5,3,1
3,04/29/2020,04/09/2020,9019,4803,1253,386
4,04/29/2020,04/03/2020,9389,5523,1688,582


Since the scope of this exploration is pertaining to the infection rate of COVID-19, the only columns that is of interest to us is the `Number_confirmed` and the `specimen_date`. According to NYC OpenData, `specimen_date` refers to *Date of specimen collection, equivalent to diagnosis date* and 
`Number_confirmed` refers to the *Count of patients tested who were confirmed to be COVID-19 cases*. 

By using these columns, we can get a sense of how the infection rate 
changed over time in NYC. To do so, we first did a little feature enginnering and extract the month and year from the `specimen_date` column. From that, we created the `specimen_month` and the `specimen_year` columns. Once we have that, we can then group the dataframe by those columns to calculate the number of infections within that period. Doing that will yield the table below.

In [41]:
covid_infections['Date'] = pd.to_datetime(covid_infections['specimen_date'], format="%m/%d/%Y", errors = 'coerce')
covid_infections['specimen_month'] = covid_infections['Date'].dt.month
covid_infections['specimen_year'] = covid_infections['Date'].dt.year

covid_infections = covid_infections[(covid_infections['specimen_year'] == 2020) | (covid_infections['specimen_year'] == 2021)]
covid_infections.dropna(inplace=True)

infect_by_month = covid_infections.groupby(['specimen_month', 'specimen_year']).agg({"Number_confirmed": "sum"})
infect_by_month

Number_confirmed
specimen_month specimen_year                  
1.0            2020.0                    12572
               2021.0                 21149582
2.0            2020.0                     2213
               2021.0                 10615918
3.0            2020.0                 36498634
               2021.0                  8961414
4.0            2020.0                 59437292
               2021.0                  4443065
5.0            2020.0                 20595985
               2021.0                   927088
6.0            2020.0                 11577931
               2021.0                   276954
7.0            2020.0                 11491519
               2021.0                   516961
8.0            2020.0                  9562310
               2021.0                   745596
9.0            2020.0                  9367084
               2021.0                   182004
10.0           2020.0                 10252869
11.0           2020.0                 14570967
12.0           2020.0                 20336188

Looking at that table, it is quite hard to tell what is going on and the trends that are occuring. With the power of plotly express, we can plot the data on to a graph and more clearly see what is going on.  

In [42]:
import plotly_express as px
import calendar
infect_2020 = infect_by_month.unstack()[('Number_confirmed', 2020)]
infect_2021 = infect_by_month.unstack()[('Number_confirmed', 2021)].fillna(0)

months = [calendar.month_name[x] + " 2020" for x in range(1, len(infect_2020) + 1) ] +\
        [calendar.month_name[x] + " 2021" for x in range(1, len(infect_2020) + 1)] 

combined_data = list(zip(infect_2020.tolist() + infect_2021.tolist(), months)) 
df = pd.DataFrame(combined_data, columns=["infections", "month"])
px.histogram(df, x="month", y="infections")


From this bar graph, we can tell that the pandemic started around March of 2020. Infections peaked during April and as we were headed towards the summertime, the amount of infections started to plateau. From the fall to the winter of 2020, we see the infections starting to rise up slowly. For 2021, it was not as much of a rollercoaster as 2020 as we just see a steady decline in the amount of 
infections over the coming months.

From a data science prospective, the period of April to July of 2020 is very interesting because we see this very sharp decline in infections. Something must have happened between that period for this decline to occur. As a resident of NYC during that time, I know that we were in one of the harshest lockdowns in the US and that maybe the lockdowns are a probable explanation for that sharp decline. To know this for us, we have to take a look at the individuals who were infected and how they were infected. 

## CDC Individual COVID-19 Infection Data
---

The CDC or Center for Disease Control and Prevention is a US national public health agency that has been monitoring and collecting information on COVID-19. This [dataset](https://data.cdc.gov/Case-Surveillance/COVID-19-Case-Surveillance-Public-Use-Data-with-Ge/n8mc-b4w4) provides information on *the demographics, geography (county and state of residence), any exposure history, disease severity indicators and outcomes, and presence of any underlying medical conditions and risk behaviors*  of individuals that were infected by COVID-19. While this dateset does not include everyone who was infected that lived in NYC, it is big enough to provide a representative sample for us to explore. Reading in the csv, we get this table:

In [43]:
nyc_individual = pd.read_csv("nyc_individual_covid_data.csv")
nyc_individual.head()

,case_month,res_state,state_fips_code,res_county,county_fips_code,age_group,sex,race,ethnicity,case_positive_specimen_interval,case_onset_interval,process,exposure_yn,current_status,symptom_status,hosp_yn,icu_yn,death_yn,underlying_conditions_yn
0,2020-12,NY,36,BRONX,36005,0 - 17 years,Female,American Indian/Alaska Native,Hispanic/Latino,0.0,0.0,Missing,Missing,Probable Case,Symptomatic,Missing,Missing,Missing,NaN
1,2021-01,NY,36,QUEENS,36081,0 - 17 years,Female,American Indian/Alaska Native,Hispanic/Latino,1.0,0.0,Missing,Missing,Laboratory-confirmed case,Symptomatic,Missing,Missing,Missing,NaN
2,2021-01,NY,36,KINGS,36047,18 to 49 years,Female,American Indian/Alaska Native,Hispanic/Latino,1.0,0.0,Missing,Missing,Probable Case,Symptomatic,Missing,Missing,Missing,NaN
3,2020-11,NY,36,RICHMOND,36085,18 to 49 years,Female,American Indian/Alaska Native,Hispanic/Latino,1.0,0.0,Missing,Missing,Probable Case,Symptomatic,Missing,Missing,Missing,NaN
4,2020-12,NY,36,BRONX,36005,0 - 17 years,Female,American Indian/Alaska Native,Hispanic/Latino,NaN,0.0,Missing,Missing,Probable Case,Symptomatic,Missing,Missing,Missing,NaN


From this dataset, we will be using the `case_month` and `exposure_yn` columns. As the CDC puts it, the `case_month` is the date the information was received by the CDC and the `exposure_yn` is described as *in the 14 days prior to illness onset, did the patient have any of the following known exposures: domestic travel, international travel, cruise ship or vessel travel as a passenger or crew member, workplace, airport/airplane, adult congregate living facility (nursing, assisted living, or long-term care facility), school/university/childcare center, correctional facility, community event/mass gathering, animal with confirmed or suspected COVID-19, other exposure, contact with a known COVID-19 case?*. By using these columns, it can help us to understand how many individuals were infected due to exposure and maybe how a lockdown can affect such exposure.

Extracting the month and year as before from `case_month`, we can group our dataframe by the month and the year and see how many individuals were infected in this dataset as well as how many were infected due to exposure. 

In [44]:
nyc_individual['month'] = nyc_individual['case_month'].apply(lambda x: int(x[5:]))
nyc_individual['year'] = nyc_individual['case_month'].apply(lambda x: int(x[:4]))
nyc_individual['underlying_conditions_yn'].fillna("No", inplace=True)

expose_nyc_individual = pd.get_dummies(nyc_individual, columns=['exposure_yn'], drop_first=True)
expose_month_data = expose_nyc_individual.groupby(['month', 'year']).agg(
    infected = pd.NamedAgg("case_month", aggfunc="count") , # number of people infected
    exposed = pd.NamedAgg("exposure_yn_Yes", aggfunc="sum") #number of people infected due to exposure
).unstack().fillna(0)

expose_month_data

infected           exposed        
year       2020      2021    2020    2021
month                                    
1          72.0  147254.0     1.0  1933.0
2         650.0   94683.0     9.0   728.0
3       58575.0  103035.0    21.0   787.0
4       99074.0   58679.0    14.0   765.0
5       24439.0   14372.0     6.0    57.0
6        9474.0    5571.0    12.0   138.0
7        8370.0   21767.0   183.0   792.0
8        6844.0   47184.0   325.0  1494.0
9       10763.0   40538.0   291.0  1021.0
10      18332.0   22225.0   338.0   377.0
11      52598.0       0.0   636.0     0.0
12     108551.0       0.0   831.0     0.0

Graphing this data:

In [45]:
cdc_infected = expose_month_data['infected', 2020].tolist() + expose_month_data['infected', 2021].tolist()
cdc_exposed = expose_month_data['exposed', 2020].tolist() + expose_month_data['exposed', 2021].tolist()

type_list = ['infected' for x in range(len(cdc_infected))] + ['exposed' for x in range(len(cdc_exposed))]
months = [calendar.month_name[x] + " 2020" for x in range(1, len(cdc_exposed) + 1) if x <= 12] +\
         [calendar.month_name[x] + " 2021" for x in range(1, len(cdc_exposed) + 1)if x <= 12] 


combined_data = list(zip(cdc_infected + cdc_exposed, months + months, type_list)) 

df = pd.DataFrame(combined_data, columns=["individuals", "month_year", "type"])
px.histogram(df, x="month_year", y="individuals", color="type" )       

Looking at the graph, we see the similar trends appear here as it did in the NYC OpenData dataset on a much smaller scale. The small red line above each bar denotes the amount of exposed individuals. As we can see, exposure contributed on a small percentage to the overall infection rate. Even if this is the case, we should explore how exposure rate changes over time to understand the effectiveness of lockdowns. To get a clearer understanding, we can take a look at the percentage of individuals who were infected due to exposure. To find the percentage, we just divide the amount of exposed individuals by the amount of infected individuals, and multiply the result by one hundred to get a percentage. Doing that will yield this table below:

In [46]:
expose_month_data['percent_exposed', 2020]= 100 * (expose_month_data['exposed'][2020]  / expose_month_data['infected'][2020])
expose_month_data['percent_exposed', 2021]= 100 * (expose_month_data['exposed'][2021]  / expose_month_data['infected'][2021])
expose_month_data.fillna(0, inplace=True)
expose_month_data['percent_exposed']

year,2020,2021
month,,
1,1.388889,1.312698
2,1.384615,0.768881
3,0.035851,0.763818
4,0.014131,1.303703
5,0.024551,0.396605
6,0.126662,2.477114
7,2.186380,3.638535
8,4.748685,3.166328
9,2.703707,2.518625


Graphing these results, we get: 

In [47]:
percent_expose_2020 = expose_month_data[('percent_exposed', 2020)].tolist()
percent_expose_2021 = expose_month_data[('percent_exposed', 2021)].tolist()

months = [calendar.month_name[x] + " 2020" for x in range(1, len(percent_expose_2020) + 1) if x <= 12] +\
         [calendar.month_name[x] + " 2021" for x in range(1, len(percent_expose_2021) + 1)if x <= 12] 

combined_data = list(zip(percent_expose_2020 + percent_expose_2021, months)) [:-2]

df = pd.DataFrame(combined_data, columns=["percent_expose", "month_year"])
px.histogram(df, x='month_year', y='percent_expose')


Looking at the graph, we see that exposure rate was at its lowest during April to June of 2020 which was during the same period that NYC was in lockdown. The exposure rate started to rise during the summer of 2020 and 2021 which makes sense as that is when people are more likely to go outside as well as the fact that lockdown measures were slowly being lifted.  

### T-test
To determine if the lockdown had a major impact, we can run something called a [t-test](https://www.investopedia.com/terms/t/t-test.asp) to determine if that period from March to June is statisically significant compared to rest of the data. To start we must first setup our null hypothesis. The null hypothesis as the name implies is the opposite of our hypothesis so in this case, our null hypotheis is that there is no statisical difference between the data found between March through June and the rest of the data. We would also need to setup our threshold or p-value, that is if the value returned from our t-test is greater than such threshold we have chosen, we can safely rejected the null hypothesis. In this case our threshold will be a p-value = 0.05 or 5%. 

Sepearting out the data and running the t-test, 

In [48]:
from scipy.stats import ttest_ind

period_expose = percent_expose_2020[2:6] #data from March to June
rest_expose = percent_expose_2020[0:2] + percent_expose_2020[6:] + percent_expose_2021[:-2]
ttest = ttest_ind(period_expose, rest_expose, equal_var=False)
statistic, pvalue = ttest
print(f"Our p-value is equal to {round(pvalue, 10)} percent.")

Our p-value is equal to 2.4486e-06 percent.


Since the p-value returned from our t-test is much lower than our threshold p-value, we can reject the null hypothesis and safely say that the data from the  period of March through June is statisically significant. While we have proven statisitical significance, it is difficult to still attribute this significance to the lockdowns due to the limitations of what the CDC defines as exposure but this is still a great start for our exploration.   

## CDC Vaccinaton Dataset
---

After the start of 2021, the COVID-19 vaccine became available to the general public. Vaccinatons were first adminstered to front-line healthcare workers as they were most at risk of getting infected and slowly became availiable to the general public. As a student of data science, it is difficult for me to speak on the effectiveness of the vaccine from a scientific point of view, but what we can still take a look at how the COVID-19 vaccine impacted the infection rate.

Luckily, the CDC provides vaccination data for the entire country and the dataset can be found [here](https://data.cdc.gov/Vaccinations/COVID-19-Vaccinations-in-the-United-States-County/8xkx-amqh). Filtering the dataset to only include NYC and using pandas to read the dataset, we get this table:


In [49]:
covid_vaccine_nyc = pd.read_csv("nyc_vaccine.csv")
covid_vaccine_nyc.dropna(inplace=True)
covid_vaccine_nyc.head()

,Date,FIPS,MMWR_week,Recip_County,Recip_State,Series_Complete_Pop_Pct,Series_Complete_Yes,Series_Complete_12Plus,Series_Complete_12PlusPop_Pct,Series_Complete_18Plus,...,SVI_CTGY,Series_Complete_Pop_Pct_SVI,Series_Complete_12PlusPop_Pct_SVI,Series_Complete_18PlusPop_Pct_SVI,Series_Complete_65PlusPop_Pct_SVI,Metro_status,Series_Complete_Pop_Pct_UR_Equity,Series_Complete_12PlusPop_Pct_UR_Equity,Series_Complete_18PlusPop_Pct_UR_Equity,Series_Complete_65PlusPop_Pct_UR_Equity
0,11/16/2021,36047,46,Kings County,NY,61.1,1564315,1564160,72.6,1467479,...,D,16.0,16.0,16.0,16.0,Metro,4.0,4.0,4.0,4.0
1,11/16/2021,36061,46,New York County,NY,74.9,1219561,1219437,83.3,1162992,...,C,12.0,12.0,12.0,12.0,Metro,4.0,4.0,4.0,4.0
2,11/16/2021,36005,46,Bronx County,NY,62.6,887632,887553,75.2,816533,...,D,16.0,16.0,16.0,16.0,Metro,4.0,4.0,4.0,4.0
3,11/16/2021,36085,46,Richmond County,NY,64.3,306292,306260,74.9,285961,...,B,8.0,8.0,8.0,8.0,Metro,4.0,4.0,4.0,4.0
4,11/16/2021,36081,46,Queens County,NY,75.1,1693016,1692929,87.1,1578493,...,C,12.0,12.0,12.0,12.0,Metro,4.0,4.0,4.0,4.0


From this dataset, the two columns that are of interest to us is the `Date` and the `Series_Complete_Yes` columns. From the CDC website, `Series_Complete_Yes` refers to the *total number of people who are fully vaccinated (have second dose of a two-dose vaccine or one dose of a single-dose vaccine) based on the jurisdiction and county where recipient lives*.

We can then group our data by the `Date` to see how many individuals were vaccinated on a specific date. If we then take our infection data from the NYC OpenData dataset, we can merge the two together. Through merging the two datasets, we get back the number of individuals that were fully vaccinated and the number of individuals infected on the same day. 

In [50]:
infect_by_day = covid_infections.groupby(['Date']).agg({"Number_confirmed": "sum"})

covid_vaccine_nyc['Date'] = pd.to_datetime(covid_vaccine_nyc['Date'], format="%m/%d/%Y")
vax_per_day = covid_vaccine_nyc.groupby(['Date']).agg({"Series_Complete_Yes": "sum"})

joined_data = vax_per_day.merge(infect_by_day, how ="left" ,on = "Date")
joined_data = joined_data[joined_data['Series_Complete_Yes'] > 0]
joined_data.dropna(inplace=True)
joined_data.head()

,Series_Complete_Yes,Number_confirmed
Date,,
2021-01-08,4140,883725.0
2021-01-09,8802,615169.0
2021-01-10,10070,516082.0
2021-01-11,10556,919924.0
2021-01-12,18608,837443.0


To visualize this table more clearly, we can plot the data points onto a scatterplot. 

In [51]:

vax_rate = joined_data['Series_Complete_Yes'].to_list()
infection_rate = joined_data['Number_confirmed'].to_list()
px.scatter(x=vax_rate,
           y=infection_rate, 
           title = "Vaccination Rate vs COVID-19 Infection Rate in NYC (2021)",
           labels={"x": "Number of vaccinated individuals (millions)",
                    "y": "Number of infected individuals (millions)"
                 })

If we look at this graph, we can see that as the number of vaccinated individuals increase, the number of infected individuals decrease. However, this trend does not look linear at all. Rather if we were to draw a line of best fit for this data, it would look more like a curve than a line. 

To me, this curve reminds me of a reciprocal function when the x values are positive. In order to test how well my curve will fit to the graph, we will use r<sup>2</sup> or the r-squared. [R-squared](https://www.investopedia.com/terms/r/r-squared.asp#limitations-of-r-squared) *is a statistical measure of fit that indicates how much variation of a dependent variable is explained by the independent variable(s) in a regression model* and is a value between zero where the higher the value, the better the fit. 

Like before, if we want to establish that there is a correlation between the infection and vaccination rate, we first need to state our null hypothesis and our threshold. Our null hypothesis is that there is no correlation between the infection and vaccination rate and for our threshold we would like to see a r-squared value greater than 0.8.

While r-squared does provide an understanding of how well our model fits the data, it does not provide an understanding of how biased our model is. In statistics, bias refers to the tendency of how a given parameter might underestimate or overestimate a value. For example, if we had a perfect model for our data, and we start to introduce new data into that model, the results might underestimate or overestimate what we expect to happen because the model was too well fitted to the initial dataset. To prevent this from happening, we split our data into a training and testing set where the training set is used to train the model and the testing set is used to test how well our model functions. In our case, we will be using 30% of the data for test and the rest of the data for training.  

In [52]:
import warnings
import numpy as np
from sklearn.metrics import r2_score
from scipy.optimize import curve_fit
from sklearn.model_selection import train_test_split 



vax_rate = joined_data['Series_Complete_Yes'].to_list()
infection_rate = joined_data['Number_confirmed'].to_list()

random_state = 40
x_train, x_test, y_train, y_test = train_test_split(vax_rate, infection_rate, test_size=0.3, random_state=random_state)

#generalized reciprocal function
def recip_func(x, a, b,c):
    return a * (1 / (b + x)) + c;

warnings.filterwarnings("ignore")
[a, b, c] , params_covar = curve_fit(recip_func,x_train,y_train)

values = [recip_func(x, a, b, c) for x in x_test];
r2score = r2_score(y_test, values)
equation = f"y = ({np.format_float_scientific(a, precision=2)} / ({np.format_float_scientific(b, precision=2)} + x)) + {np.format_float_scientific(c, precision=2)}"
print(f"The r2 score for this curve fitting is {r2score}")
print(f" The equation of the curve is {equation}")

The r2 score for this curve fitting is 0.9103803016466594
 The equation of the curve is y = (5.38e+11 / (7.28e+05 + x)) + -8.54e+04


Plotting this curve onto our scatterplot, we get

In [53]:
import plotly.graph_objects as go


#creating the regression line
myline = np.linspace(0, max(vax_rate), len(vax_rate))
y_pred = [recip_func(x, a, b, c) for x in myline];


#Plotting the regression line and data
layout = dict(
            title = "Vaccination Rate vs COVID-19 Infection Rate in NYC (2021)",
            xaxis=dict(title='Number of vaccinated individuals (millions)'),
            yaxis=dict(title='Number of infected individuals (millions)')
            )


fig = go.Figure(layout=layout)

#scatter plot
fig.add_trace(go.Scatter(
                x=vax_rate, 
                y=infection_rate, 
                mode='markers', 
                showlegend = False,  
            ))

#line 
fig.add_trace(go.Scatter(x=myline, y=y_pred, mode= 'lines', showlegend=False))
fig

Here our r-squared value is around 0.91 which means that 91% of the variance in our infectoin rate data can be explained by the vaccination data. This shows that these two factors are highly correlating.  

However that is only for one iteration. Since the training and testing datasets are picked arbitrarily, we can go through our dataset a certain amount of iterations to see on average how well our model fits the data. In this case, lets try a 1000 iterations and we get,  

In [13]:
scores = []
iterations = 1000
for i in range(iterations):
    x_train, x_test, y_train, y_test = train_test_split(vax_rate, infection_rate, test_size=0.3)
    def func(x, a, b,c):
        return a * (1 / (b + x)) + c;


    warnings.filterwarnings("ignore")
    [a, b, c] , params_covar = curve_fit(func,x_train, y_train)
    values = [func(x, a, b, c) for x in x_test];
    r2score = r2_score(y_test, values)
    if r2score > 0:
        scores.append(r2score)


print(f"The average r2 score is {np.mean(scores).round(decimals = 4)}"+ f" with a standard deviation of {np.std(scores).round(decimals =2)} for {iterations} iterations")

The average r2 score is 0.8559 with a standard deviation of 0.15 for 1000 iterations


## Conclusions and Final Thoughts
---

Through looking at these three datasets from two different sources, we have found very interesting insight with regards to the COVID-19 and NYC. From this basic inquiry, we have seen a statistically significant period between March through June of 2020 in regards to those who have been infected through exposure. 

As we continue to look at NYC COVID-19 infections, another topic of interest came about which were the vacccines. Using the CDC and NYC OpenData datasets on vaccinated and infected indivduals, we built and tested a model to determine if such a correlation existed and indeed a correlation does exist. 

From these basic findings, a lot more interesting questions arise. For example, the definition of exposure as defined by the CDC is extrememly specific and if we were to broaden the scope of the definition, would we still see this signficance? Like any statisical inqiury, the rabbit hole can always get deeper.